## Working with data

PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the Dataset.

- `Dataset` 比作是**图书馆的书架**，里面每一本书（样本）都有自己的编号（索引）和标签（类别）
- `DataLoader` 比作**图书馆的书车**，每次从书架（Dataset）中取出一批书（batch）并交给你。

总结：
- Dataset：定义和存储数据，包含所有的样本及其标签。
- DataLoader：批量加载数据，支持多线程并行加载和数据随机化。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

`torchvision.datasets`中包含许多来自现实世界中的数据，这里使用一个名为 **FashionMNIST**的数据集，其中包含 2 个参数

- transform 用于修改数据集中的样本
- target_transform 用于修改标签

In [2]:
# 从公开数据集中下载训练数据
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从公开数据集中下载测试数据集
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)


00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:16<00:00, 1585958.72it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 30334448.07it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:00<00:00, 11399502.36it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 4119877.31it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Fashion-MNIST 数据集包含的是影像数据，其中有 60,000 个训练集和 10,000个测试集，每一个样本都是由一张 28x28 像素的灰度图片以及一个标签组成。

- root="data"：数据集的存储位置，下载的文件会存放在 data/ 文件夹中。
- train=True：指定加载的是训练集。
- download=True：如果本地的 data/ 目录中不存在 FashionMNIST 数据集，则自动从互联网下载。
- transform=ToTensor()：在加载图像时，将其从PIL图像转换为PyTorch张量，并将像素值从[0, 255]缩放到[0, 1]。

We pass the `Dataset` as an argument to `DataLoader`. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

将 Dataset 传递给 DataLoader，对数据完成自动批次化、采样，洗牌等

In [3]:
batch_size = 64

# 创建 data loader
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f'X 的形状为: {X.shape}')
    print(f'y 的形状为: {y.shape} {y.dtype}')
    break

X 的形状为: torch.Size([64, 1, 28, 28])
y 的形状为: torch.Size([64]) torch.int64


---
## Creating Models
To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU or MPS if available.

为了定义一个神经网络，在此创建一个继承nn.Module的类。在这个类中，通过__init__函数定义神经网络的层，并且定义forward 函数来告诉数据如何在网络中传递。

学习如何构建神经网络，[link](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

In [4]:
# 创建模型
# 确定使用的是什么设备，cpu gpu 还是 mps，用于确定哪些设备可以用来加速运算
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'可使用 {device} 设备！')

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # 确保 PyTorch 正确初始化了 nn.Module 的内部结构
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

可使用 cpu 设备！
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
为了训练模型，需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)和一个[优化器](https://pytorch.org/docs/stable/optim.html)

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单次训练过程（epoch）中，模型会在训练数据集上做出预测，并反向转播预测误差用于调整模型的参数

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # 将模型设置为训练模式，启动 dropout 和 batch normalization
    for batch, (X, y) in enumerate(dataloader): # 遍历每个批次的输入数据 X 和对应的标签 y
        X, y = X.to(device), y.to(device) # 将数据转移到相应的设备上，主要用于显卡加速训练

        # 计算预测误差
        pred = model(X) # 前向传播，模型对 X 的预测
        loss = loss_fn(pred, y) # 计算预测值和真实标签之间的损失

        # 反向传播
        loss.backward() # 计算损失函数对模型参数的梯度
        optimizer.step() # 使用优化器更新模型的参数
        optimizer.zero_grad() # 清除优化器中的梯度，否则梯度会累积

        if batch % 100 == 0: # 每 100 个批次，打印一次训练状态
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'损失: {loss:>7f} [{current:>5d}/{size:>5d}]')

接下来在测试数据集上检测模型的性能，来判断模型是不是正在学习

In [7]:
def test(dataloader, model, loss_fn): # 测试函数相比于训练函数少了一个优化器
    size = len(dataloader.dataset) # 测试集的样本总量
    num_batches = len(dataloader) # 总的批次数量
    model.eval() # 将模型设置为“评估模式”
    test_loss, correct = 0, 0 # 初始化测试损失和正确预测的数量
    with torch.no_grad(): # 禁用梯度跟踪，测试过程不需要反向传播和梯度计算
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # 将数据加载到设备上
            pred = model(X) # 前向传播，使用模型预测 X 的结果
            test_loss += loss_fn(pred, y).item() # 计算批次的损失 并进行批次间的损失累加
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 求得正确预测的样本数量
    test_loss /= num_batches # 计算平均损失
    correct /= size # 计算准确率
    print(f'测试错误: \n 准确度: {(100*correct):>0.1f}%, 平均损失: {test_loss:>8f}\n')

接下来，让我们训练模型，经过每一次 epoch 后，可以看到模型的一个学习情况。

In [8]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1}\n-----------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    print('完成!')

Epoch 1
-----------------------------------
损失: 2.294090 [   64/60000]
损失: 2.278678 [ 6464/60000]
损失: 2.266753 [12864/60000]
损失: 2.267455 [19264/60000]
损失: 2.234830 [25664/60000]
损失: 2.209408 [32064/60000]
损失: 2.212016 [38464/60000]
损失: 2.171990 [44864/60000]
损失: 2.177907 [51264/60000]
损失: 2.150384 [57664/60000]
测试错误: 
 准确度: 47.0%, 平均损失: 2.135348

完成!
Epoch 2
-----------------------------------
损失: 2.142908 [   64/60000]
损失: 2.126368 [ 6464/60000]
损失: 2.070242 [12864/60000]
损失: 2.092798 [19264/60000]
损失: 2.028975 [25664/60000]
损失: 1.970496 [32064/60000]
损失: 1.995919 [38464/60000]
损失: 1.909347 [44864/60000]
损失: 1.919222 [51264/60000]
损失: 1.844375 [57664/60000]
测试错误: 
 准确度: 54.2%, 平均损失: 1.840108

完成!
Epoch 3
-----------------------------------
损失: 1.877382 [   64/60000]
损失: 1.836685 [ 6464/60000]
损失: 1.721901 [12864/60000]
损失: 1.768942 [19264/60000]
损失: 1.651521 [25664/60000]
损失: 1.611743 [32064/60000]
损失: 1.633813 [38464/60000]
损失: 1.537056 [44864/60000]
损失: 1.561803 [51264/60000]
损失: 1

## Saving Models
保存模型的常见方法是序列化内部状态字典（包含模型参数）

In [9]:
torch.save(model.state_dict(), 'model.pth')
print('保存模型状态为 model.pth')

保存模型状态为 model.pth


## Loading Models
首先重新构建一个初始化状态的模型，然后加载模型的权重，即可恢复到训练后的模型状态。

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth', weights_only=True))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 评估
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'预测: "{predicted}", 实际: "{actual}"')

预测: "Ankle boot", 实际: "Ankle boot"
